In [118]:
import numpy as np
import torch

In [119]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [120]:
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [121]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [122]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [124]:
def gd(weight,bias,inputs,targets,epochs=1000):
    for i in range(epochs):
        preds = model(inputs)
        loss = mse(preds, targets)
        loss.backward()
        with torch.no_grad():
            weight -= w.grad * 1e-5
            bias -= b.grad * 1e-5
            weight.grad.zero_()
            bias.grad.zero_()
    return preds,loss

In [123]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-1.1643, -2.1830,  0.4433],
        [ 0.4144, -1.1673,  2.6518]], requires_grad=True)
tensor([0.9168, 0.1356], requires_grad=True)


In [125]:
preds,loss = gd(w,b,inputs,targets,epochs=1700)
print(loss)
print(preds)
print(targets)

tensor(11.4644, grad_fn=<DivBackward0>)
tensor([[ 57.1482,  69.9754],
        [ 83.5531, 103.6294],
        [115.6616, 126.7650],
        [ 20.2215,  35.2806],
        [104.9464, 125.3218]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
